### Importing Libraries

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

### Pre-Processing Location Column

In [2]:
# Load the dataframe
df = pd.read_excel('CleanedData.xlsx')

In [3]:
df.head()

,Date,UserName,Tweet,Location,IsVerified_True,extracted_hashtags,clean_tweet,tokenized_tweet,lemmatized_tweet,stemmed_tweet
0,12/1/22,mazen160,@bitquark I am looking forward for a ChatGPT A...,At a server near you.,0,[],looking forward api lot tools built,"['looking', 'forward', 'api', 'lot', 'tools', ...","['looking', 'forward', 'api', 'lot', 'tools', ...","['look', 'forward', 'api', 'lot', 'tool', 'bui..."
1,12/1/22,ZoumanaCisse6,i think i broke chatGPT. it's been frozen for ...,Bay area,0,[],think broke frozen couple minutes,"['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'coupl', 'minut']"
2,12/1/22,DennisLibre,"I like ChatGPT as much as the next guy, but th...",Miami,0,[],like guy theres twitter im looking guy tweets ...,"['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'there', 'twitter', 'im', 'loo..."
3,12/1/22,mpaepper,"It seems like sth. was changed here, because I...","Hamburg, Germany",0,[],like sth changed replicate behavior refuses wr...,"['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'chang', 'replic', 'behavior',..."
4,12/1/22,AaronSitze,I know ChatGPT is just crawling for inputs. \n...,"Illinois, USA",0,[],know crawling inputs nneven makes sad vision e...,"['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawl', 'input', 'nneven', 'make', '..."


In [4]:
df.shape

(716810, 10)

In [5]:
df.isnull().sum()

Date                      0
UserName                  0
Tweet                     0
Location                  0
IsVerified_True           0
extracted_hashtags        0
clean_tweet           18796
tokenized_tweet           0
lemmatized_tweet          0
stemmed_tweet             0
dtype: int64

In [6]:
df = df.dropna(how= "any").copy()

In [7]:
df.isnull().sum()

Date                  0
UserName              0
Tweet                 0
Location              0
IsVerified_True       0
extracted_hashtags    0
clean_tweet           0
tokenized_tweet       0
lemmatized_tweet      0
stemmed_tweet         0
dtype: int64

In [8]:
df['Location'].value_counts()

United States        11567
India                10170
London, England       7334
San Francisco, CA     7033
New York, NY          6882
                     ...  
No Regrets               1
Noida, India...          1
Bloggosfera              1
dvlan buhgz ty6ie        1
The Blastzone            1
Name: Location, Length: 91316, dtype: int64

In [9]:
df_new = pd.DataFrame(df)

# reset the index
df_new = df_new.reset_index()
df_new.drop(columns = ['index'])

# print the dataframe
df_new.head()

,index,Date,UserName,Tweet,Location,IsVerified_True,extracted_hashtags,clean_tweet,tokenized_tweet,lemmatized_tweet,stemmed_tweet
0,0,12/1/22,mazen160,@bitquark I am looking forward for a ChatGPT A...,At a server near you.,0,[],looking forward api lot tools built,"['looking', 'forward', 'api', 'lot', 'tools', ...","['looking', 'forward', 'api', 'lot', 'tools', ...","['look', 'forward', 'api', 'lot', 'tool', 'bui..."
1,1,12/1/22,ZoumanaCisse6,i think i broke chatGPT. it's been frozen for ...,Bay area,0,[],think broke frozen couple minutes,"['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'coupl', 'minut']"
2,2,12/1/22,DennisLibre,"I like ChatGPT as much as the next guy, but th...",Miami,0,[],like guy theres twitter im looking guy tweets ...,"['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'there', 'twitter', 'im', 'loo..."
3,3,12/1/22,mpaepper,"It seems like sth. was changed here, because I...","Hamburg, Germany",0,[],like sth changed replicate behavior refuses wr...,"['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'chang', 'replic', 'behavior',..."
4,4,12/1/22,AaronSitze,I know ChatGPT is just crawling for inputs. \n...,"Illinois, USA",0,[],know crawling inputs nneven makes sad vision e...,"['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawl', 'input', 'nneven', 'make', '..."


### Extracting Countries from Location Column.

In [10]:
from geopy.exc import GeocoderTimedOut
# Create a geolocator object
geolocator = Nominatim(user_agent='my-app')

# Define a function to get the country name
def get_country(location, retries=3):
    try:
        # Use geolocator to get the location object
        loc = geolocator.geocode(location, timeout=10)

        # Use the location object to get the country name
        return loc.raw['display_name'].split(',')[-1].strip()
    except GeocoderServiceError as e:
        # Retry up to the specified number of times, with an increasing delay between retries
        if retries > 0:
            delay = 2 ** (3 - retries)
            print(f"Geocoding failed with error '{e}', retrying in {delay} seconds...")
            time.sleep(delay)
            return get_country(location, retries=retries-1)
        else:
            # Return None if the maximum number of retries has been reached
            print(f"Geocoding failed with error '{e}', maximum number of retries reached")
            return None
    except (AttributeError, GeocoderTimedOut) as e:
        # Return None if the location cannot be geocoded
        print(f"Geocoding failed with error '{e}'")
        return None

#### Executed the countries batch for every 5000 rows.

In [ ]:
# Initialize the starting index and the batch size
start_idx = 4508
batch_size = 500

# Loop until the end of the dataframe is reached or the maximum number of records is processed
while start_idx < len(df) and start_idx < 5000:
    # Get the ending index of the current batch
    end_idx = start_idx + batch_size
    
    # Apply the function to the current batch and assign the results to the 'country' column
    df_new.loc[start_idx:end_idx, 'country'] = df_new.loc[start_idx:end_idx, 'index'].apply(get_country)
    
    # Increment the starting index to the beginning of the next batch
    start_idx = end_idx + 1

    # Print a message to indicate the progress
    print(f"Processed {end_idx} records")

In [11]:
df_new = pd.read_csv('df.csv')

In [12]:
df_new = df_new.rename(columns = {"Location": 'Value_counts', 'index' : 'Location'})

In [13]:
df_new.head()

,Location,Value_counts,country
0,United States,11785,United States
1,India,10515,India
2,"London, England",7474,United Kingdom
3,"San Francisco, CA",7111,United States
4,"New York, NY",6970,United States


In [14]:
df_new.shape

(94548, 3)

In [15]:
df_new.to_csv('Countries.csv', index = False)

In [16]:
df_countries = pd.read_csv('Countries.csv')

In [17]:
df_countries.country.value_counts()

United States            1501
United Kingdom            344
India                     245
Canada                    208
Deutschland               127
                         ... 
Pacific Ocean               1
Crna Gora / Црна Гора       1
Gabon                       1
ދިވެހިރާއްޖެ                1
Guyana                      1
Name: country, Length: 174, dtype: int64

In [18]:
df_countries.head()

,Location,Value_counts,country
0,United States,11785,United States
1,India,10515,India
2,"London, England",7474,United Kingdom
3,"San Francisco, CA",7111,United States
4,"New York, NY",6970,United States


### Merging countries to main dataframe

In [19]:
df = df.merge(df_countries, how = 'left', on = 'Location')

In [20]:
df.head()

,Date,UserName,Tweet,Location,IsVerified_True,extracted_hashtags,clean_tweet,tokenized_tweet,lemmatized_tweet,stemmed_tweet,Value_counts,country
0,12/1/22,mazen160,@bitquark I am looking forward for a ChatGPT A...,At a server near you.,0,[],looking forward api lot tools built,"['looking', 'forward', 'api', 'lot', 'tools', ...","['looking', 'forward', 'api', 'lot', 'tools', ...","['look', 'forward', 'api', 'lot', 'tool', 'bui...",1.0,NaN
1,12/1/22,ZoumanaCisse6,i think i broke chatGPT. it's been frozen for ...,Bay area,0,[],think broke frozen couple minutes,"['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'coupl', 'minut']",19.0,United States
2,12/1/22,DennisLibre,"I like ChatGPT as much as the next guy, but th...",Miami,0,[],like guy theres twitter im looking guy tweets ...,"['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'there', 'twitter', 'im', 'loo...",374.0,United States
3,12/1/22,mpaepper,"It seems like sth. was changed here, because I...","Hamburg, Germany",0,[],like sth changed replicate behavior refuses wr...,"['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'chang', 'replic', 'behavior',...",291.0,Deutschland
4,12/1/22,AaronSitze,I know ChatGPT is just crawling for inputs. \n...,"Illinois, USA",0,[],know crawling inputs nneven makes sad vision e...,"['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawl', 'input', 'nneven', 'make', '...",329.0,United States


In [21]:
df.drop(columns = ['Value_counts'], inplace = True)

In [22]:
df.head()

,Date,UserName,Tweet,Location,IsVerified_True,extracted_hashtags,clean_tweet,tokenized_tweet,lemmatized_tweet,stemmed_tweet,country
0,12/1/22,mazen160,@bitquark I am looking forward for a ChatGPT A...,At a server near you.,0,[],looking forward api lot tools built,"['looking', 'forward', 'api', 'lot', 'tools', ...","['looking', 'forward', 'api', 'lot', 'tools', ...","['look', 'forward', 'api', 'lot', 'tool', 'bui...",NaN
1,12/1/22,ZoumanaCisse6,i think i broke chatGPT. it's been frozen for ...,Bay area,0,[],think broke frozen couple minutes,"['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'couple', 'minutes']","['think', 'broke', 'frozen', 'coupl', 'minut']",United States
2,12/1/22,DennisLibre,"I like ChatGPT as much as the next guy, but th...",Miami,0,[],like guy theres twitter im looking guy tweets ...,"['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'theres', 'twitter', 'im', 'lo...","['like', 'guy', 'there', 'twitter', 'im', 'loo...",United States
3,12/1/22,mpaepper,"It seems like sth. was changed here, because I...","Hamburg, Germany",0,[],like sth changed replicate behavior refuses wr...,"['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'changed', 'replicate', 'behav...","['like', 'sth', 'chang', 'replic', 'behavior',...",Deutschland
4,12/1/22,AaronSitze,I know ChatGPT is just crawling for inputs. \n...,"Illinois, USA",0,[],know crawling inputs nneven makes sad vision e...,"['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawling', 'inputs', 'nneven', 'make...","['know', 'crawl', 'input', 'nneven', 'make', '...",United States


In [23]:
df.shape

(698014, 11)

In [24]:
df.country.value_counts()

United States         209297
United Kingdom         44511
India                  41024
Canada                 26610
Australia              15447
                       ...  
Северна Македонија        15
Brunei                    14
Guyana                    14
Lesotho                   13
اليمن                     12
Name: country, Length: 170, dtype: int64

In [25]:
df.country.isna().sum()

232635

In [26]:
df.country.fillna('Unknown', inplace = True)

In [27]:
(df.isnull().sum()* 100 / len(df)).round(2).sort_values(ascending = False)

Date                  0.0
UserName              0.0
Tweet                 0.0
Location              0.0
IsVerified_True       0.0
extracted_hashtags    0.0
clean_tweet           0.0
tokenized_tweet       0.0
lemmatized_tweet      0.0
stemmed_tweet         0.0
country               0.0
dtype: float64

In [28]:
df.to_excel('Preprocessed_data.xlsx', index=False)